In [ ]:
#ali code
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException
import pandas as pd
import time

driver = webdriver.Chrome()
driver.maximize_window()
wait = WebDriverWait(driver, 20)

# Navigate to Facebook login page
url = 'http://www.facebook.com'
driver.get(url)

# Target username and password fields
username = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "input[name='email']")))
password = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "input[name='pass']")))

# Enter credentials
username.clear()
username.send_keys("aroobaminhas14@gmail.com")
password.clear()
password.send_keys("beautyinsights") 

# Click login button
button = WebDriverWait(driver, 2).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button[type='submit']"))).click()
time.sleep(5)  

page_url = 'https://www.facebook.com/groups/superwomenofpakistan/'
driver.get(page_url)
time.sleep(5)  

all_posts = []

for _ in range(50):
    try:
        posts = driver.find_elements(By.XPATH, "//*[contains(@data-ad-comet-preview, 'message') and contains(@data-ad-preview, 'message')]")

        for post in posts:
            try:
                # Check and click the "See more" button if it exists
                try:
                    see_more_button = post.find_element(By.XPATH, ".//div[contains(text(), 'See more')]")
                    driver.execute_script("arguments[0].click();", see_more_button)
                    time.sleep(1) 
                except NoSuchElementException:
                    pass

                # Collect all inner div text elements within each post
                post_text_parts = post.find_elements(By.XPATH, ".//div[@style='text-align: start;']")
                post_text = " ".join([part.text for part in post_text_parts if part.text]) 

                # Extract the post ID
                try:
                    post_id = post.get_attribute("id")  # Directly access the 'id' attribute
                except NoSuchElementException:
                    post_id = None

                if post_text:
                    all_posts.append({'Post': post_text, 'Post ID': post_id})
            except StaleElementReferenceException:
                continue
    
    except StaleElementReferenceException:
        continue

    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(3) 

df_posts = pd.DataFrame(all_posts)
df_posts.to_csv('facebook_page_posts_with_ids.csv', index=False)
print("Posts and Post IDs have been saved to facebook_page_posts_with_ids.csv")

driver.quit()


Text Extration parsing into table form

In [ ]:
import re
import csv

# Path to the extracted text file
service_text_file = "extracted_text5.txt"

def parse_services(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        content = file.read()

    services = re.split(r'--- Text from .* ---', content)
    parsed_services = []

    for service in services:
        if service.strip():
            lines = service.strip().split("\n")
            service_provider = lines[0] if lines else "Unknown Service Provider"

            details = " ".join(lines[1:])
            # Extract deal number
            deal_match = re.search(r'DEAL\s+\d+', details, re.IGNORECASE)
            deal = deal_match.group() if deal_match else "Unknown Deal"
            
            # Extract all prices
            price_matches = re.findall(r'Rs\.\s*\d{1,3}(?:,\d{3})*(?:\.\d+)?/-', details)
            price = ", ".join(price_matches) if price_matches else "Price not specified"
            
            # Extract services offered (all text before the first price occurrence)
            price_index = details.find(price_matches[0]) if price_matches else len(details)
            services_offered = details[:price_index].strip()

            parsed_services.append({
                "Service_Provider": service_provider.strip(),
                "Deal": deal.strip(),
                "Services_Offered": services_offered.strip(),
                "Price": price.strip(),
                "Details": details.strip()
            })

    return parsed_services

services_data = parse_services(service_text_file)

csv_file = "parsed_services.csv"
csv_columns = ["Service_Provider", "Deal", "Services_Offered", "Price", "Details"]

try:
    with open(csv_file, 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=csv_columns)
        writer.writeheader()
        writer.writerows(services_data)
    print(f"Data successfully written to {csv_file}")
except IOError as e:
    print(f"Error writing to CSV file: {e}")


Data successfully written to parsed_services.csv


In [ ]:
import pandas as pd
file_path = 'parsed_services.csv' 
df = pd.read_csv(file_path)
print("Displaying the entire DataFrame:")
df.head()

Displaying the entire DataFrame:


,Service_Provider,Deal,Services_Offered,Price,Details
0,AIMS BEAUTY SALON,DEAL 39,DEAL 39 HAIR DEAL Base Color Highlights Lowlig...,"Rs.10,000/-, Rs.12,000/-",DEAL 39 HAIR DEAL Base Color Highlights Lowlig...
1,NSMVA SALON,DEAL 41,DEAL 41 HYDRAFACIAL Manicure Pedicure Eyebrows...,"Rs. 4,999/-",DEAL 41 HYDRAFACIAL Manicure Pedicure Eyebrows...
2,AIMS BEAUTY SALON,DEAL 38,DEAL 38 HAIR DEAL Fashion Dye Hair Cut Blow Dr...,"Rs. 6,000/-",DEAL 38 HAIR DEAL Fashion Dye Hair Cut Blow Dr...
3,SARA SALON,DEAL 37,DEAL 37 FULL BODY MASSAGE Head Massage Deep Co...,"Rs. 1,499/-",DEAL 37 FULL BODY MASSAGE Head Massage Deep Co...
4,NIMS BEAUTY SALON,DEAL 36,DEAL 36 SKIN DEAL Teen Age Cleansing Whitening...,"Rs. 4,500/-",DEAL 36 SKIN DEAL Teen Age Cleansing Whitening...


Pre processing the Facebook Post data and matching it with our service details

In [ ]:
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk

services_df = pd.read_csv("parsed_services.csv")
posts_df = pd.read_csv("facebook_page_posts_with_ids.csv")

def clean_text(text):
    # Remove special characters, numbers, and convert to lowercase
    text = re.sub(r"[^a-zA-Z\s]", "", str(text))  # Keep only letters and spaces
    text = text.lower()  # Convert to lowercase
    # Tokenize and remove stopwords
    tokens = word_tokenize(text)
    stop_words = set(stopwords.words("english"))
    tokens = [word for word in tokens if word not in stop_words]
    return " ".join(tokens)

posts_df["Cleaned_Post"] = posts_df["Post"].apply(clean_text)

# Step 2: Match posts with keywords, keeping only the first match
leads = []

# Extract keywords from 'Services_Offered' column
keywords = services_df["Services_Offered"].str.lower().str.split().explode().unique()

# Match posts with keywords, stopping after the first match
for index, row in posts_df.iterrows():
    post = row["Cleaned_Post"]  
    post_id = row["Post ID"]   
    for _, service_row in services_df.iterrows():
        if any(keyword in post for keyword in keywords):  # Check for keyword match
            leads.append({
                "Post": row["Post"],
                "Post ID": post_id,
                "Service_Provider": service_row["Service_Provider"],
                "Deal": service_row["Deal"]
            })
            break  # Stop further matching for this post

# Step 3: Save results to leads file
leads_df = pd.DataFrame(leads)
leads_file_path = "leads.csv"
leads_df.to_csv(leads_file_path, index=False)

# Step 4: Save processed posts to a new file
processed_file_path = "processed_facebook_posts.csv"
posts_df.to_csv(processed_file_path, index=False)

print(f"Processed posts saved to: {processed_file_path}")
print(f"Leads file saved to: {leads_file_path}")


Processed posts saved to: processed_facebook_posts.csv
Leads file saved to: leads.csv


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Muham\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Muham\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
import pandas as pd
file_path = 'leads.csv' 
df = pd.read_csv(file_path)
print("Displaying the entire DataFrame:")
df

Displaying the entire DataFrame:


,Post,Post ID,Service_Provider,Deal
0,Presenting our hot seller ludo / chess and co...,:r89:,AIMS BEAUTY SALON,DEAL 39
1,Post - 1 Frosted Sunlight https://chat.whats...,:ra6:,AIMS BEAUTY SALON,DEAL 39
2,I need to buy nighty and undergarments for my ...,:rb2:,AIMS BEAUTY SALON,DEAL 39
3,Thanks a ton everyone for your input. I am now...,:rc0:,AIMS BEAUTY SALON,DEAL 39
4,Just regretting my decision Social media p hy...,:re4:,AIMS BEAUTY SALON,DEAL 39
...,...,...,...,...
106,Assalamualaikum all.. This is Mahwish Adil own...,:r84a:,AIMS BEAUTY SALON,DEAL 39
107,Post#2 Launching My New Handmade Home decor ...,:r857:,AIMS BEAUTY SALON,DEAL 39
108,Mujhe is tarhan k frames bnwane henn in differ...,:r865:,AIMS BEAUTY SALON,DEAL 39
109,Has anybody selling these kind of namaz chadars?,:r883:,AIMS BEAUTY SALON,DEAL 39


Lead and Non Lead: Leads saved to refined_leads_with_ids

In [ ]:
import pandas as pd
import re
from sklearn.feature_extraction.text import CountVectorizer

# Load the datasets
services_file_path = 'parsed_services.csv'  # Replace with your actual file path
comments_file_path = 'facebook_page_posts_with_ids.csv'  # Replace with your actual file path

services_data = pd.read_csv(services_file_path)
comments_data = pd.read_csv(comments_file_path)

# Step 1: Extract Keywords from Services Data
# Combine all text from "Services_Offered" and "Details"
services_text = (
    services_data['Services_Offered'].astype(str).tolist()
    + services_data['Details'].astype(str).tolist()
)

# Extract unique keywords
services_keywords = set()
for text in services_text:
    words = re.findall(r'\b\w+\b', text.lower())  # Extract words ignoring punctuation
    services_keywords.update(words)

# Filter keywords: Remove generic/stopwords and short words
stopwords = set(['000', '1', '2', '10', 'deal', 'price', 'details', 'the', 'of', 'and', 'for', 'on', 'a', 'in', 'is'])
refined_keywords = [word for word in services_keywords if word not in stopwords and len(word) > 2]

# Step 2: Preprocess Comments Data
def preprocess_text(text):
    return re.sub(r'http\S+|www.\S+|[^\w\s]', '', text.lower())  # Remove links, special chars, and lowercase

comments_data['Cleaned_Post'] = comments_data['Post'].astype(str).apply(preprocess_text)

# Step 3: Identify Leads
def identify_leads(post, keywords):
    # Check if any of the keywords appear in the cleaned post
    return any(keyword in post for keyword in keywords)

comments_data['Lead'] = comments_data['Cleaned_Post'].apply(lambda post: 'Lead' if identify_leads(post, refined_keywords) else 'Non-Lead')

# Step 4: NLP Enhancement (Optional): Using Bag-of-Words Matching for Improved Intent Detection
vectorizer = CountVectorizer(vocabulary=refined_keywords)
keywords_matrix = vectorizer.fit_transform(comments_data['Cleaned_Post'])
comments_data['Lead_Score'] = keywords_matrix.sum(axis=1)  # Score based on keyword frequency

# Step 5: Reclassify Comments with Lead_Score = 0
comments_data['Lead'] = comments_data.apply(
    lambda row: 'Non-Lead' if row['Lead_Score'] == 0 else row['Lead'], axis=1
)

# Step 6: Apply Threshold for Lead_Score
confidence_threshold = 2  # Minimum score for confident classification
comments_data['Lead'] = comments_data.apply(
    lambda row: 'Non-Lead' if row['Lead_Score'] < confidence_threshold else row['Lead'], axis=1
)

# Step 7: Filter Leads and Export Results
leads_data = comments_data[comments_data['Lead'] == 'Lead']
output_leads_path = 'refined_leads_with_ids.csv'
leads_data[['Post ID', 'Post', 'Cleaned_Post', 'Lead', 'Lead_Score']].to_csv(output_leads_path, index=False)

# Step 8: Export Non-Leads and Potential Review Comments
review_data = comments_data[(comments_data['Lead'] == 'Non-Lead') & (comments_data['Lead_Score'] > 0)]
review_output_path = 'potential_review_comments.csv'
review_data[['Post ID', 'Post', 'Cleaned_Post', 'Lead', 'Lead_Score']].to_csv(review_output_path, index=False)

# Output Messages
print(f"Refined leads saved to {output_leads_path}")
print(f"Potential review comments saved to {review_output_path}")


Refined leads saved to refined_leads_with_ids.csv
Potential review comments saved to potential_review_comments.csv


In [ ]:
import pandas as pd
file_path = 'refined_leads_with_ids.csv' 
df = pd.read_csv(file_path)
print("Displaying the entire DataFrame:")
df

Displaying the entire DataFrame:


,Post ID,Post,Cleaned_Post,Lead,Lead_Score
0,:r3k:,Looking for interne for the following position...,looking for interne for the following position...,Lead,3
1,:r9j:,Clearance SALE Khaddar Stitched 2pc Shirt and...,clearance sale khaddar stitched 2pc shirt and...,Lead,2
2,:rci:,Hi Mommies! We are hiring at Kumon DHA Phase 8...,hi mommies we are hiring at kumon dha phase 8 ...,Lead,3
3,:rf1:,I want 3 to 4 family (parents and babyboy) pic...,i want 3 to 4 family parents and babyboy pictu...,Lead,4
4,:ri7:,I have a functions coming up and I want to go ...,i have a functions coming up and i want to go ...,Lead,6
...,...,...,...,...,...
78,:r7m9:,order now at Seemalite instagram.com/seemalit...,order now at seemalite instagramcomseemalite2...,Lead,4
79,:r7rr:,Unlock Your Earning Potential Through Social M...,unlock your earning potential through social m...,Lead,7
80,:r851:,Assalamu Alaikum. I am looking for a female ph...,assalamu alaikum i am looking for a female pho...,Lead,2
81,:r88b:,Season end sale with massive discouts on our p...,season end sale with massive discouts on our p...,Lead,6


In [ ]:
# from selenium import webdriver
# from selenium.webdriver.common.by import By
# from selenium.webdriver.support.ui import WebDriverWait
# from selenium.webdriver.support import expected_conditions as EC
# from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException
# import pandas as pd
# import time

# # Load the leads CSV
# leads_df = pd.read_csv("refined_leads_with_ids.csv")
# leads_dict = dict(zip(leads_df['Post ID'], leads_df['Post']))

# # Initialize WebDriver
# driver = webdriver.Chrome()
# driver.maximize_window()
# wait = WebDriverWait(driver, 20)

# # Navigate to Facebook login page
# url = 'http://www.facebook.com'
# driver.get(url)

# # Target username and password fields
# username = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "input[name='email']")))
# password = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "input[name='pass']")))

# # Enter credentials
# username.clear()
# username.send_keys("aroobaminhas14@gmail.com")
# password.clear()
# password.send_keys("beautyinsights")

# # Click login button
# button = WebDriverWait(driver, 2).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button[type='submit']"))).click()
# time.sleep(5)

# # Navigate to the group/page URL
# page_url = 'https://www.facebook.com/groups/superwomenofpakistan/'
# driver.get(page_url)
# time.sleep(5)

# # Extract posts
# all_posts = []

# for _ in range(20):  # Adjust scroll count as needed
#     try:
#         posts = driver.find_elements(By.XPATH, "//*[contains(@data-ad-comet-preview, 'message') and contains(@data-ad-preview, 'message')]")

#         for post in posts:
#             try:
#                 # Check and click "See more" if present
#                 try:
#                     see_more_button = post.find_element(By.XPATH, ".//div[contains(text(), 'See more')]")
#                     driver.execute_script("arguments[0].click();", see_more_button)
#                     time.sleep(1)
#                 except NoSuchElementException:
#                     pass

#                 # Collect post text and ID
#                 post_text_parts = post.find_elements(By.XPATH, ".//div[@style='text-align: start;']")
#                 post_text = " ".join([part.text for part in post_text_parts if part.text])
#                 post_id = post.get_attribute("id")

#                 if post_text and post_id:
#                     all_posts.append({'Post': post_text, 'Post ID': post_id})

#                     # Match post ID with leads data
#                     if post_id in leads_dict:
#                         comment_text = f"Hi, we noticed your post: '{leads_dict[post_id]}'. We’d love to assist you with our services!"

#                         # Scroll to the post
#                         driver.execute_script("arguments[0].scrollIntoView(true);", post)
#                         time.sleep(1)

#                         try:
#                             # Locate and click the "Comment" button
#                             comment_button = post.find_element(By.XPATH, ".//span[@data-ad-rendering-role='comment_button']")
#                             driver.execute_script("arguments[0].click();", comment_button)
#                             time.sleep(2)

#                             # Locate the comment box after clicking the "Comment" button
#                             comment_box = post.find_element(By.XPATH, ".//div[@aria-label='Write an answer…' and @contenteditable='true']")
#                             comment_box.click()  # Ensure focus on the text box
#                             time.sleep(1)
#                             comment_box.send_keys(comment_text)
#                             comment_box.send_keys("\n")  # Press Enter to post the comment
#                             print(f"Commented on post {post_id}: {comment_text}")

#                         except NoSuchElementException:
#                             print(f"Failed to comment on post {post_id}: Comment button or text box not found")

#             except StaleElementReferenceException:
#                 continue

#     except StaleElementReferenceException:
#         continue

#     # Scroll to load more posts
#     driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
#     time.sleep(3)

# # Save posts to a CSV for reference
# df_posts = pd.DataFrame(all_posts)
# df_posts.to_csv('reference.csv', index=False)
# print("Posts and Post IDs have been saved to facebook_page_posts_with_ids.csv")

# # Quit the driver
# driver.quit()
